In [1]:
import sys
import numpy as np
import import_ipynb
from scipy.sparse.linalg import spsolve
from pathlib import Path

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from pre_processing import create_domain, read_mesh
    from fem_processing import boundary_conditions
    from pos_processing import graph_results
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")
    
#run ../setup_project.py

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!
Modules imports were successful!
Modules imports were successful!


# Project 1: Poisson Problem at L Domain 

# 10-noded Cubic Triangular Elements, $P_3$
## Pre-processor module
### Geometry and mesh Domain

In [2]:
FINITE_ELEMENT = ("Triangle", 3)
BOUNDARY = [{'type': 'Dirichlet', 'tag': 101, 'value': 0.0, 'name': 'entire_boundary'}]
MATERIAL = [{'type': 'material_constant_a', 'tag': 201, 'value': 1, 'name': 'entire_domain'}]

# Create mesh from file geometry
create_domain.L_geometry(FINITE_ELEMENT, BOUNDARY, lc = 1 / (2**3), view_mesh=True)
mesh_data = read_mesh.get_data(FINITE_ELEMENT, BOUNDARY, MATERIAL, model='L')
graph_results.plot_mesh(mesh_data, FINITE_ELEMENT, model='L')

Model L_domain_Triangle3 (2D)
Info     : 13 geometric entities
Info     : 2 Physical Groups
Info     : 4327 nodes in total
Info     : 934 2-D elements in total
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p13_L_domain_poisson\pre_processing\pictures\L_meshed_domain_Triangle3.svg


### Figure 1: L meshed domain

<figure>
    <img src="pre_processing/pictures/L_meshed_domain_Triangle3.svg" alt="Fig.1" style="width:100%;" />
    <figcaption>Figure 1: Rectangular meshed domain $\Omega=[0,1]^2$.</figcaption>
</figure>

In [3]:
nodes = [node['global_coord'] for node in mesh_data['nodes_data']]
conn = [element['conn_list'] for element in mesh_data['conn_data']]

#### `apply_physics()`

In [4]:
mesh_data = ps.apply_physics(mesh_data, FINITE_ELEMENT)
mesh_data['conn_data'][:1]

[{'TagID': 83,
  'conn_list': [440, 292, 547, 674, 675, 676, 677, 678, 679, 680],
  'material': {'type': 'material_constant_a',
   'tag': 201,
   'value': 1,
   'name': 'entire_domain',
   'a_value': 1},
  'source': {'type': 'analytical_source_fx', 'value': 0}}]

### Imposition of Boundary Conditions

In [5]:
dirichlet_nodes = [
    node['TagID'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
dirichlet_values = [
    node['bc']['value'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Dirichlet']
free_nodes = [
    node['TagID'] for node in mesh_data['nodes_data'] if node['bc']['type'] == 'Free']

print(f"The entire domain has {len(nodes)} nodes: " 
      f"{len(nodes) - len(dirichlet_nodes)} free nodes and {len(dirichlet_nodes)} Dirichlet nodes.")

The entire domain has 4327 nodes: 4081 free nodes and 246 Dirichlet nodes.


### `assign_dirichlet_potential()`

In [6]:
potential_function = lambda x, y: ps.potential_and_gradient(x, y)[0]
boundary_conditions.assign_dirichlet_potential(mesh_data, potential_function)

### Reduced Global matrix $A_{gr}$ with boundary conditions

In [7]:
Agr, fgr = boundary_conditions.get_reduced_global_matrix(mesh_data, FINITE_ELEMENT)
print("Reduced global matrix shape:", Agr.shape)

Reduced global matrix shape: (4081, 4081)


## Processor Module
### Reduced Global matrix

In [8]:
ur = spsolve(Agr.tocsr(), fgr.toarray())
potential_u = boundary_conditions.get_global_potentials(mesh_data, ur)

## Post-Processor

In [9]:
graph_results.approximate_fem_solution(FINITE_ELEMENT, mesh_data, potential_u, model='L')

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p13_L_domain_poisson\pos_processing\pictures\L_domain_solution_Triangle3.svg


### Figure 2: Scalar Field Distribution on a Numeric Triangular Mesh
<figure>
    <img src="pos_processing/pictures/L_domain_solution_Triangle3.svg" alt="Fig.2" style="width:100%;" />
    <figcaption>Figure 2: Post-Processor: Scalar Field Distribution.</figcaption>
</figure>

### Figure 3: Analytical Scalar Field Distribution
<figure>
    <img src="pos_processing/pictures/L_analytical_solution.svg" alt="Fig.3" style="width:100%;" />
    <figcaption>Figure 4: Analytical Scalar Field Distribution on a Triangular Mesh.</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.